In [1]:
import pandas as pd

In [2]:
file = 'diabetic_data.csv'
df = pd.read_csv(file)

In [4]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        101766 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      101766 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  101766 non-null object
medical_specialty           101766 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            10176

In [19]:
colnames = df.columns.tolist()

In [25]:
# Show the unique values for each column as first step to identifying how missing values are shown, issues with binary columns, etc.
for name in colnames:
    print(name, df[name].unique())

encounter_id [  2278392    149190     64410 ... 443854148 443857166 443867222]
patient_nbr [  8222157  55629189  86047875 ... 140199494 120975314 175429310]
race ['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
gender ['Female' 'Male' 'Unknown/Invalid']
age ['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
weight ['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
admission_type_id [6 1 2 3 4 5 8 7]
discharge_disposition_id [25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]
admission_source_id [ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]
time_in_hospital [ 1  3  2  4  5 13 12  9  7 10  6 11  8 14]
payer_code ['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
medical_specialty ['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedic

In [26]:
# How many missing values in race?
df.race.value_counts()

Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

In [27]:
# How many missing or unknown values in gender?
df.gender.value_counts()

Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64

In [28]:
# How many missing or unknown values in weight?
df.weight.value_counts()

?            98569
[75-100)      1336
[50-75)        897
[100-125)      625
[125-150)      145
[25-50)         97
[0-25)          48
[150-175)       35
[175-200)       11
>200             3
Name: weight, dtype: int64

In [29]:
# How many missing or unknown values in payer code:
df.payer_code.value_counts()

?     40256
MC    32439
HM     6274
SP     5007
BC     4655
MD     3532
CP     2533
UN     2448
CM     1937
OG     1033
PO      592
DM      549
CH      146
WC      135
OT       95
MP       79
SI       55
FR        1
Name: payer_code, dtype: int64

In [37]:
# To understand each variable better, create a table showing the names of the columns, data type, 
# first row's values, and descriptions from the data dictionary

data_dictionary = pd.read_csv('diabetes-data-dict.csv')


In [38]:
data_dictionary.head()

,Feature name,Type,Description and values,% missing
0,Encounter ID,Numeric,Unique identifier of an encounter,0%
1,Patient number,Numeric,Unique identifier of a patient,0%
2,Race,Nominal,"Values: Caucasian, Asian, African American, Hi...",2%
3,Gender,Nominal,"Values: male, female, and unknown/invalid",0%
4,Age,Nominal,"Grouped in 10-year intervals: 0, 10), 10, 20),...",0%


In [39]:
data_dictionary.tail()

,Feature name,Type,Description and values,% missing
23,A1c test result,Nominal,Indicates the range of the result or if the te...,0%
24,Change of medications,Nominal,Indicates if there was a change in diabetic me...,0%
25,Diabetes medications,Nominal,Indicates if there was any diabetic medication...,0%
26,24 features for medications,Nominal,"For the generic names: metformin, repaglinide,...",0%
27,Readmitted,Nominal,Days to inpatient readmission. Values: â€œ<30â...,0%


In [68]:
# expand row 26 so each medication is on a separate row
drugs = data_dictionary.iloc[26][2]
drugs

'For the generic names: metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone, the feature indicates whether the drug was prescribed or there was a change in the dosage. Values: â€œupâ€\x9d if the dosage was increased during the encounter, â€œdownâ€\x9d if the dosage was decreased, â€œsteadyâ€\x9d if the dosage did not change, and â€œnoâ€\x9d if the drug was not prescribed'

In [69]:
# import regex library
import re

In [75]:
drug_list = re.compile('For the generic names: (.+?), the').findall(drugs)

In [76]:
drug_list

['metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone']

In [79]:
drug_list2 = drug_list[0].split(', ')

In [80]:
len(drug_list2)


23

In [81]:
type(drug_list2)

list

In [82]:
drug_list2[-1] = 'metformin-pioglitazone'

In [83]:
drug_list2[-1]

'metformin-pioglitazone'

In [105]:
#insert the list into the dataframe replacing row 26 with the list of 23 drugs - use concat?
df_drug = pd.DataFrame({'Feature name':drug_list2})
df_drug.head()

,Feature name
0,metformin
1,repaglinide
2,nateglinide
3,chlorpropamide
4,glimepiride


In [112]:
df_drug['Type'] = 'Nominal'

In [117]:
data_dict = pd.concat(data_dictionary.iloc[:26], df_drug).reset_index(drop=True)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [118]:
data_dict

,% missing,Description and values,Feature name,Type,0
0,0%,Unique identifier of an encounter,Encounter ID,Numeric,NaN
1,0%,Unique identifier of a patient,Patient number,Numeric,NaN
2,2%,"Values: Caucasian, Asian, African American, Hi...",Race,Nominal,NaN
3,0%,"Values: male, female, and unknown/invalid",Gender,Nominal,NaN
4,0%,"Grouped in 10-year intervals: 0, 10), 10, 20),...",Age,Nominal,NaN
5,97%,Weight in pounds.,Weight,Numeric,NaN
6,0%,Integer identifier corresponding to 9 distinct...,Admission type,Nominal,NaN
7,0%,Integer identifier corresponding to 29 distinc...,Discharge disposition,Nominal,NaN
8,0%,Integer identifier corresponding to 21 distinc...,Admission source,Nominal,NaN
9,0%,Integer number of days between admission and d...,Time in hospital,Numeric,NaN


In [104]:
df.loc[0]

encounter_id                                 2278392
patient_nbr                                  8222157
race                                       Caucasian
gender                                        Female
age                                           [0-10)
weight                                             ?
admission_type_id                                  6
discharge_disposition_id                          25
admission_source_id                                1
time_in_hospital                                   1
payer_code                                         ?
medical_specialty           Pediatrics-Endocrinology
num_lab_procedures                                41
num_procedures                                     0
num_medications                                    1
number_outpatient                                  0
number_emergency                                   0
number_inpatient                                   0
diag_1                                        

In [119]:
df.loc[10]

encounter_id                          28236
patient_nbr                        89869032
race                        AfricanAmerican
gender                               Female
age                                 [40-50)
weight                                    ?
admission_type_id                         1
discharge_disposition_id                  1
admission_source_id                       7
time_in_hospital                          9
payer_code                                ?
medical_specialty                         ?
num_lab_procedures                       47
num_procedures                            2
num_medications                          17
number_outpatient                         0
number_emergency                          0
number_inpatient                          0
diag_1                                250.7
diag_2                                  403
diag_3                                  996
number_diagnoses                          9
max_glu_serum                   